In [49]:
import pandas as pd, datetime, urllib.request, json
from tkinter import Tk, ttk, StringVar, messagebox
from tkcalendar import DateEntry
from datetime import datetime, date
from dateutil import parser

std_date_format = '%d/%m/%Y'

In [50]:
def GUI(START_DATE = datetime(1995, 1, 1), END_DATE = datetime.now(), PERIOD = 'DAY', CAPITAL = '0', save_csv_i = False, save_xlsx_i = False):
    window = Tk()
    window.title('Giant Steps Internship Test')
    window.resizable(False, False)
    window.eval('tk::PlaceWindow . center')
    window.call('wm', 'attributes', '.', '-topmost', True)
    
    START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL = START_DATE, END_DATE, PERIOD, CAPITAL
    save_csv, save_xlsx = save_csv_i, save_xlsx_i
    
    START_DATE_ENTRY = DateEntry(justify = 'center', mindate=START_DATE, date_pattern='dd/mm/y')
    START_DATE_ENTRY.set_date('01/01/1995')
    START_DATE_LABEL = ttk.Label(text = 'Insert the start date:', justify = 'center')
    
    END_DATE_ENTRY = DateEntry(justify = 'center', maxdate=END_DATE, date_pattern='dd/mm/y')
    END_DATE_ENTRY.set_date(END_DATE.strftime(std_date_format))
    END_DATE_LABEL = ttk.Label(text = 'Insert the end date:', justify = 'center')
    
    PERIOD_VAR = StringVar(window)
    PERIOD_VAR.set(PERIOD)
    PERIOD_LABEL = ttk.Label(text = 'Select the frequency:', justify = 'center')
    period_list = ['DAY', 'MONTH', 'YEAR']
    PERIOD_ENTRY = ttk.OptionMenu(window, PERIOD_VAR, PERIOD, *period_list)
    
    CAPITAL_ENTRY = ttk.Entry(justify = 'center', exportselection = 0, width=10)
    CAPITAL_ENTRY.insert(-1, CAPITAL)
    CAPITAL_LABEL = ttk.Label(text = 'Insert the initial capital amount:', justify = 'center')
    
    def isClickedFunction():
        nonlocal START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL
        
        START_DATE_FINAL = START_DATE_ENTRY.get()
        END_DATE_FINAL = END_DATE_ENTRY.get()
        PERIOD_FINAL = PERIOD_VAR.get()
        CAPITAL_FINAL = CAPITAL_ENTRY.get()
    
        window.destroy()
    
    def save_csvfunc():
        nonlocal save_csv
        save_csv = True
        
    def save_xlsxfunc():
        nonlocal save_xlsx
        save_xlsx = True
    
    BUTTON = ttk.Button(text = 'Search', command = isClickedFunction)
    
    save_csv_var = StringVar(value=0)
    save_xlsx_var = StringVar(value=0)
    
    save_csv = ttk.Checkbutton(window, text='Save .csv file', command=save_csvfunc, variable=save_csv_var)
    save_xlsx = ttk.Checkbutton(window, text='Save .xlsx file', command=save_xlsxfunc, variable=save_xlsx_var)
 
    PERIOD_LABEL.grid(row=1, column=1, pady=10)
    PERIOD_ENTRY.grid(row=2, column=1, padx=25)
    
    START_DATE_LABEL.grid(row=3, column=0, pady=10)
    START_DATE_ENTRY.grid(row=4, column=0, padx=25)
    
    END_DATE_LABEL.grid(row=3, column=2, pady=10)
    END_DATE_ENTRY.grid(row=4, column=2, padx=25)
    
    CAPITAL_LABEL.grid(row=3, column=1, pady=10)
    CAPITAL_ENTRY.grid(row=4, column=1, padx=25)
    
    BUTTON.grid(row=5, column=1, pady=15)
    
    save_csv.grid(row=5, column=0, pady=15)
    save_xlsx.grid(row=5, column=2, pady=15)
    
    window.mainloop()
    
    return START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL, save_csv, save_xlsx

In [51]:
START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL, save_csv, save_xlsx = GUI()
CAPITAL_FINAL = float(CAPITAL_FINAL)

while True:
    if parser.parse(START_DATE_FINAL) > parser.parse(END_DATE_FINAL):
        root = Tk()
        root.withdraw()
        messagebox.showerror("Error", "End date must not exceed start date")
        root.destroy()
        START_DATE_FINAL, END_DATE_FINAL, PERIOD_FINAL, CAPITAL_FINAL, save_csv, save_xlsx = GUI(PERIOD = PERIOD_FINAL, CAPITAL = CAPITAL_FINAL, save_csv_i = save_csv, save_xlsx_i = save_xlsx)
    else:
        break

In [52]:
initial_day = START_DATE_FINAL.split('/')[0]
initial_month = START_DATE_FINAL.split('/')[1]
initial_year = START_DATE_FINAL.split('/')[2]

final_day = END_DATE_FINAL.split('/')[0]
final_month = END_DATE_FINAL.split('/')[1]
final_year = END_DATE_FINAL.split('/')[2]

In [53]:
selic_rates_base_url = r'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial=dd/mm/yy&dataFinal=dd/mm/yy'
selic_rates_url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial={initial_day}/{initial_month}/{initial_year}&dataFinal={final_day}/{final_month}/{final_year}'

with urllib.request.urlopen(selic_rates_url) as url:
    data = json.loads(url.read().decode())
    selic_rates = pd.DataFrame(data)
    
selic_rates['data'] = pd.to_datetime(selic_rates['data'], format = std_date_format).dt.strftime(std_date_format)

In [54]:
selic_rates

,data,valor
0,30/12/1994,0.160667
1,02/01/1995,0.154000
2,03/01/1995,0.153333
3,04/01/1995,0.153333
4,05/01/1995,0.153333
...,...,...
6882,02/06/2022,0.047279
6883,03/06/2022,0.047279
6884,06/06/2022,0.047279
6885,07/06/2022,0.047279
